# Desenvolvendo um modelo de aprendizado de máquina para prever o valor de mercado de um carro.

# Contents <a id='back'></a>

* [Introdução](#intro)
* [Etapa 1. Visão geral dos dados](#data_review)
* [Etapa 2. Transformações dos dados](#tranforamation_data)
* [Etapa 3. Modelo](#model)
* [Conclusões](#end)
    

## Introdução <a id='intro'></a>

O serviço de vendas de carros usados Rusty Bargain está desenvolvendo um aplicativo para atrair novos clientes. Nesse aplicativo, você pode descobrir rapidamente o valor de mercado do seu carro. Você tem acesso a dados históricos: especificações técnicas, versões de acabamento e preços. Você precisa construir o modelo para determinar o valor. 

Rusty Bargain está interessado em:

- a qualidade da predição;
- a velocidade da predição;
- o tempo necessário para o treinamento

### Objetivo: 
- Você seguiu todos os passos das instruções?
- Como você preparou os dados?
- Quais modelos e hiperparâmetros você considerou?
- Você conseguiu evitar a duplicação de código?
- Quais são suas conclusões?
- Você manteve a estrutura do projeto?
- Você manteve o código limpo?


### Descrição de dados

- `Abra e examine o arquivo de dados. Caminho para o arquivo:`
/datasets/car_data.csv. Baixar o conjunto de dados

[Voltar ao Índice](#back)

## Etapa 1. Visão geral dos dados <a id='data_review'></a>

Carregue os dados das consultas, explore-os e carregue bibliotecas que você acredita que são necessárias para o projeto.

In [1]:
# Importa as bibliotecas necessárias
import pandas as pd  # Utilizada para manipulação de dados em DataFrames
import numpy as np  # Utilizada para trabalhar com arrays e operações numéricas
import matplotlib.pyplot as plt  # Utilizada para criar gráficos e visualizações
from scipy import stats as st  # Importa a função de estatísticas da biblioteca SciPy e dá um apelido como 'st'
import warnings  # Utilizada para controlar a exibição de avisos no código

# Ignora os avisos que possam aparecer durante a execução do código
warnings.filterwarnings("ignore")


## Carregue os Dados

In [2]:
# Carregue o arquivo com os dados em um DataFrame
df = pd.read_csv('C:/Users/gabri/Downloads/Corrigido/Projeto 12/car_data.csv')

In [3]:
# vamos exibir as primeiras 5 linhas
df.head(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [4]:
# Vamos ver quantas linhas e colunas nosso conjunto de dados tem
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

### Explore os dados iniciais



#### Descrição de dados

- `DateCrawled` — data em que o perfil foi baixado do banco de dados
- `VehicleType` — tipo de carroçaria do veículo
- `RegistrationYear` — ano de matrícula do veículo
- `Gearbox` — tipo de caixa de transmissão
- `Power` — potência (hp)
- `Model` — modelo do veículo
- `Mileage` — quilometragem (medida em km devido às especificidades regionais do conjunto de dados)
- `RegistrationMonth` — mês de registro do veículo
- `FuelType` — tipo de combustível
- `Brand` — marca do veículo
- `NotRepaired` — veículo reparado ou não
- `DateCreated` — data de criação do perfil
- `NumberOfPictures` — número de fotos do veículo
- `PostalCode` — código postal do proprietário do perfil (usuário)
- `LastSeen` — data da última atividade do usuário

Objetivo

- `Price` — preço (Euro)

Agora vamos explorar nossos dados. Você vai querer ver quantas colunas e linhas ele tem, veja algumas linhas para verificar possíveis problemas com os dados.

### Duplicatas <a id='duplicates'></a>
Encontrando de duplicatas óbvias na tabela:

In [5]:
# Verificar Valores Duplicados
df.duplicated().sum()

262

In [6]:
# Aborde as duplicatas
df = df.drop_duplicates().reset_index(drop=True)

In [7]:
# Verificar Valores Duplicados
df.duplicated().sum()

0

### Valores ausentes <a id='missing_values'></a>

In [8]:
# Verificar Valores Ausentes
df.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37484
RegistrationYear         0
Gearbox              19830
Power                    0
Model                19701
Mileage                  0
RegistrationMonth        0
FuelType             32889
Brand                    0
NotRepaired          71145
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

In [9]:
# Vamos calcular a porcentagem desses valores faltantes em relação com o nosso conjunto de dados
(df.isnull().sum() / df.shape[0]) * 100

DateCrawled           0.000000
Price                 0.000000
VehicleType          10.585501
RegistrationYear      0.000000
Gearbox               5.600002
Power                 0.000000
Model                 5.563573
Mileage               0.000000
RegistrationMonth     0.000000
FuelType              9.287871
Brand                 0.000000
NotRepaired          20.091385
DateCreated           0.000000
NumberOfPictures      0.000000
PostalCode            0.000000
LastSeen              0.000000
dtype: float64

Removemos os dados duplicados e analizamos os valores faltantes. 

## Etapa 2. Transformação de dados <a id='tranforamation_data'></a>
Vamos examinar cada coluna para ver quais problemas podemos ter nelas.

In [10]:
df['DateCrawled'].sort_values()

44600     01/04/2016 00:06
266156    01/04/2016 00:06
157525    01/04/2016 00:06
119430    01/04/2016 00:10
122927    01/04/2016 00:25
                ...       
78331     31/03/2016 23:58
344885    31/03/2016 23:58
102481    31/03/2016 23:58
59088     31/03/2016 23:58
201452    31/03/2016 23:58
Name: DateCrawled, Length: 354107, dtype: object

In [11]:
df['Price'].describe()

count    354107.000000
mean       4416.433287
std        4514.338584
min           0.000000
25%        1050.000000
50%        2700.000000
75%        6400.000000
max       20000.000000
Name: Price, dtype: float64

In [12]:
df['VehicleType'].sort_values().unique()

array(['bus', 'convertible', 'coupe', 'other', 'sedan', 'small', 'suv',
       'wagon', nan], dtype=object)

In [13]:
df['RegistrationYear'].sort_values().unique()

array([1000, 1001, 1039, 1111, 1200, 1234, 1253, 1255, 1300, 1400, 1500,
       1600, 1602, 1688, 1800, 1910, 1915, 1919, 1920, 1923, 1925, 1927,
       1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938,
       1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950,
       1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961,
       1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972,
       1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983,
       1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994,
       1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
       2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019, 2066, 2200, 2222, 2290, 2500, 2800, 2900, 3000,
       3200, 3500, 3700, 3800, 4000, 4100, 4500, 4800, 5000, 5300, 5555,
       5600, 5900, 5911, 6000, 6500, 7000, 7100, 7500, 7800, 8000, 8200,
       8455, 8500, 8888, 9000, 9229, 9450, 9996, 99

In [14]:
df['Gearbox'].sort_values().unique()

array(['auto', 'manual', nan], dtype=object)

In [15]:
df['Power'].describe()

count    354107.000000
mean        110.089651
std         189.914972
min           0.000000
25%          69.000000
50%         105.000000
75%         143.000000
max       20000.000000
Name: Power, dtype: float64

In [16]:
df['Model'].sort_values().unique()

array(['100', '145', '147', '156', '159', '1_reihe', '1er', '200',
       '2_reihe', '300c', '3_reihe', '3er', '4_reihe', '500', '5_reihe',
       '5er', '601', '6_reihe', '6er', '7er', '80', '850', '90', '900',
       '9000', '911', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a8',
       'a_klasse', 'accord', 'agila', 'alhambra', 'almera', 'altea',
       'amarok', 'antara', 'arosa', 'astra', 'auris', 'avensis', 'aveo',
       'aygo', 'b_klasse', 'b_max', 'beetle', 'berlingo', 'bora',
       'boxster', 'bravo', 'c1', 'c2', 'c3', 'c4', 'c5', 'c_klasse',
       'c_max', 'c_reihe', 'caddy', 'calibra', 'captiva', 'carisma',
       'carnival', 'cayenne', 'cc', 'ceed', 'charade', 'cherokee',
       'citigo', 'civic', 'cl', 'clio', 'clk', 'clubman', 'colt', 'combo',
       'cooper', 'cordoba', 'corolla', 'corsa', 'cr_reihe', 'croma',
       'crossfire', 'cuore', 'cx_reihe', 'defender', 'delta', 'discovery',
       'doblo', 'ducato', 'duster', 'e_klasse', 'elefantino', 'eos',
       'escort', 'espac

In [17]:
df['Mileage'].describe()

count    354107.000000
mean     128211.811684
std       37906.590101
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: Mileage, dtype: float64

In [18]:
df['RegistrationMonth'].sort_values().unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

In [19]:
df['FuelType'].sort_values().unique()

array(['cng', 'electric', 'gasoline', 'hybrid', 'lpg', 'other', 'petrol',
       nan], dtype=object)

In [20]:
df['Brand'].sort_values().unique()

array(['alfa_romeo', 'audi', 'bmw', 'chevrolet', 'chrysler', 'citroen',
       'dacia', 'daewoo', 'daihatsu', 'fiat', 'ford', 'honda', 'hyundai',
       'jaguar', 'jeep', 'kia', 'lada', 'lancia', 'land_rover', 'mazda',
       'mercedes_benz', 'mini', 'mitsubishi', 'nissan', 'opel', 'peugeot',
       'porsche', 'renault', 'rover', 'saab', 'seat', 'skoda', 'smart',
       'sonstige_autos', 'subaru', 'suzuki', 'toyota', 'trabant',
       'volkswagen', 'volvo'], dtype=object)

In [21]:
df['NotRepaired'].sort_values().unique()

array(['no', 'yes', nan], dtype=object)

In [22]:
df['DateCreated'].sort_values()

247165    01/02/2016 00:00
293735    01/02/2016 00:00
281833    01/02/2016 00:00
201856    01/03/2016 00:00
296916    01/03/2016 00:00
                ...       
319455    31/03/2016 00:00
55814     31/03/2016 00:00
185358    31/03/2016 00:00
185419    31/03/2016 00:00
218578    31/03/2016 00:00
Name: DateCreated, Length: 354107, dtype: object

In [23]:
df['NumberOfPictures'].describe()

count    354107.0
mean          0.0
std           0.0
min           0.0
25%           0.0
50%           0.0
75%           0.0
max           0.0
Name: NumberOfPictures, dtype: float64

In [24]:
df['PostalCode'].sort_values().unique()

array([ 1067,  1068,  1069, ..., 99994, 99996, 99998], dtype=int64)

In [25]:
df['LastSeen'].sort_values()

311229    01/04/2016 00:15
300128    01/04/2016 00:15
270029    01/04/2016 00:15
84894     01/04/2016 00:15
327905    01/04/2016 00:15
                ...       
15495     31/03/2016 23:47
231071    31/03/2016 23:48
36469     31/03/2016 23:50
262530    31/03/2016 23:51
72795     31/03/2016 23:54
Name: LastSeen, Length: 354107, dtype: object

As colunas 'PostalCode' , 'LastSeen' , 'DateCrawled' e 'DateCreated', possuem valorem que não servem para nossa pesquisa por isso iremos retira-las. As colunas 'price' e 'power', possui alguns valores anormais, nos iremos retirar esses valores

### Pré-processar dados 

In [26]:
# a lista dos nomes das colunas na tabela df
df.columns

Index(['DateCrawled', 'Price', 'VehicleType', 'RegistrationYear', 'Gearbox',
       'Power', 'Model', 'Mileage', 'RegistrationMonth', 'FuelType', 'Brand',
       'NotRepaired', 'DateCreated', 'NumberOfPictures', 'PostalCode',
       'LastSeen'],
      dtype='object')

In [27]:
# renomeando colunas
df = df.rename(str.lower, axis='columns')

In [28]:
# a lista dos nomes das colunas na tabela df
df.columns

Index(['datecrawled', 'price', 'vehicletype', 'registrationyear', 'gearbox',
       'power', 'model', 'mileage', 'registrationmonth', 'fueltype', 'brand',
       'notrepaired', 'datecreated', 'numberofpictures', 'postalcode',
       'lastseen'],
      dtype='object')

In [29]:
#Deixando as colunas que realmentes influenciam nossa pesquisa
df = df[['price','vehicletype','registrationyear','gearbox',
       'power', 'model', 'mileage', 'registrationmonth', 'fueltype', 'brand',
       'notrepaired']]

In [30]:
#Retirando valores do Mês que correspondem a 0
df = df.query("registrationmonth > 0")

In [31]:
#Subistituindo valores NAN de 'notrepaired' para 'unknownrepaired'
df['notrepaired'].fillna('unknownrepaired',inplace=True )

In [32]:
#Retirando outliers da coluna 'price' e power
Q1_price = df['price'].quantile(0.25)
Q3_price =df['price'].quantile(0.75)
IQR_price = Q3_price - Q1_price
limite_superior_price = Q3_price+(1.5*IQR_price)
limite_inferior_price = Q1_price - (1.5 * IQR_price)

In [33]:
Q1_power = df['power'].quantile(0.25)
Q3_power =df['power'].quantile(0.75)
IQR_power = Q3_power - Q1_power
limite_superior_power = Q3_power+(1.5*IQR_power)
limite_inferior_power = Q1_power - (1.5 * IQR_power)


In [34]:
df_final = df[df['price'] > limite_inferior_price] 
df_final = df_final[df_final['price'] < limite_superior_price]
df_final = df_final[df_final['power'] > limite_inferior_power] 
df_final = df_final[df_final['power'] < limite_superior_power]

In [35]:
df_final['price'].describe()

count    296234.000000
mean       4016.808277
std        3648.408706
min           0.000000
25%        1200.000000
50%        2790.000000
75%        5950.000000
max       15249.000000
Name: price, dtype: float64

In [36]:
#valores abaixo do outliers sobstituindo com a média em preço
dfpriceok = df_final.query("price > @Q1_price ")
dfpriceok = dfpriceok.groupby(['vehicletype','registrationyear','gearbox','model'])['price'].mean()

In [37]:
#valores abaixo do outliers sobstituindo com a média em potência
dfpowerok = df_final.query("power > @Q1_power ")
dfpowerok = dfpowerok.groupby(['vehicletype','registrationyear','gearbox','model'])['price'].mean()

In [38]:
def fill_price(row):
    if row['price'] <  Q1_price :
        return dfpriceok[row['price']]
    return row['price']


In [39]:
def fill_power(row):
    if row['power'] <  Q1_power :
        return dfpowerok[row['power']]
    return row['power']

In [40]:
df_final['price'] = df_final.apply(fill_price, axis=1)
df_final['power'] = df_final.apply(fill_power, axis=1)

In [41]:
df_final['price'].describe()

count    296234.000000
mean       5690.894652
std        3647.273691
min        1250.000000
25%        2500.000000
50%        4812.500000
75%        7999.000000
max       15249.000000
Name: price, dtype: float64

In [42]:
df_final['power'].describe()

count    296234.000000
mean       1258.166519
std        2429.866596
min          75.000000
25%         105.000000
50%         140.000000
75%         606.750000
max       12400.000000
Name: power, dtype: float64

In [43]:
(df_final.isnull().sum() / df_final.shape[0]) * 100

price                0.000000
vehicletype          7.405632
registrationyear     0.000000
gearbox              3.319335
power                0.000000
model                4.146384
mileage              0.000000
registrationmonth    0.000000
fueltype             6.270718
brand                0.000000
notrepaired          0.000000
dtype: float64

In [44]:
#como os valores dos valores faltantes são umastra pequena de todos os dados, por isso estão retirando
df_final.dropna(inplace=True)

In [45]:
(df_final.isnull().sum() / df_final.shape[0]) * 100

price                0.0
vehicletype          0.0
registrationyear     0.0
gearbox              0.0
power                0.0
model                0.0
mileage              0.0
registrationmonth    0.0
fueltype             0.0
brand                0.0
notrepaired          0.0
dtype: float64

In [46]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 251133 entries, 2 to 354106
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   price              251133 non-null  float64
 1   vehicletype        251133 non-null  object 
 2   registrationyear   251133 non-null  int64  
 3   gearbox            251133 non-null  object 
 4   power              251133 non-null  float64
 5   model              251133 non-null  object 
 6   mileage            251133 non-null  int64  
 7   registrationmonth  251133 non-null  int64  
 8   fueltype           251133 non-null  object 
 9   brand              251133 non-null  object 
 10  notrepaired        251133 non-null  object 
dtypes: float64(2), int64(3), object(6)
memory usage: 23.0+ MB


In [47]:
df_final

,price,vehicletype,registrationyear,gearbox,power,model,mileage,registrationmonth,fueltype,brand,notrepaired
2,9800.0,suv,2004,auto,163.000000,grand,125000,8,gasoline,jeep,unknownrepaired
3,1500.0,small,2001,manual,75.000000,golf,150000,6,petrol,volkswagen,no
4,3600.0,small,2008,manual,2566.333333,fabia,90000,7,gasoline,skoda,no
5,4900.0,sedan,1995,manual,102.000000,3er,150000,10,petrol,bmw,yes
6,2200.0,convertible,2004,manual,109.000000,2_reihe,150000,8,petrol,peugeot,no
...,...,...,...,...,...,...,...,...,...,...,...
354100,3200.0,sedan,2004,manual,225.000000,leon,150000,5,petrol,seat,yes
354101,14990.0,bus,2000,manual,8000.000000,zafira,150000,3,petrol,opel,no
354104,12950.0,convertible,2000,auto,101.000000,fortwo,125000,3,petrol,smart,no
354105,9200.0,bus,1996,manual,102.000000,transporter,150000,3,gasoline,volkswagen,no


In [48]:
(df_final.isnull().sum() / df_final.shape[0]) * 100

price                0.0
vehicletype          0.0
registrationyear     0.0
gearbox              0.0
power                0.0
model                0.0
mileage              0.0
registrationmonth    0.0
fueltype             0.0
brand                0.0
notrepaired          0.0
dtype: float64

In [49]:
df_final = df_final.reset_index(drop=True)

In [50]:
df_final

,price,vehicletype,registrationyear,gearbox,power,model,mileage,registrationmonth,fueltype,brand,notrepaired
0,9800.0,suv,2004,auto,163.000000,grand,125000,8,gasoline,jeep,unknownrepaired
1,1500.0,small,2001,manual,75.000000,golf,150000,6,petrol,volkswagen,no
2,3600.0,small,2008,manual,2566.333333,fabia,90000,7,gasoline,skoda,no
3,4900.0,sedan,1995,manual,102.000000,3er,150000,10,petrol,bmw,yes
4,2200.0,convertible,2004,manual,109.000000,2_reihe,150000,8,petrol,peugeot,no
...,...,...,...,...,...,...,...,...,...,...,...
251128,3200.0,sedan,2004,manual,225.000000,leon,150000,5,petrol,seat,yes
251129,14990.0,bus,2000,manual,8000.000000,zafira,150000,3,petrol,opel,no
251130,12950.0,convertible,2000,auto,101.000000,fortwo,125000,3,petrol,smart,no
251131,9200.0,bus,1996,manual,102.000000,transporter,150000,3,gasoline,volkswagen,no


Outliers removidos podemos prosseguir para construção do modelo.

## Etapa 3. Modelo <a id='model'></a>

In [51]:
# Importa os módulos necessários do scikit-learn (biblioteca para aprendizado de máquina)
from sklearn.model_selection import train_test_split  # Função para dividir os dados em conjuntos de treino e teste
from sklearn.ensemble import RandomForestClassifier  # Algoritmo de aprendizado de máquina para classificação baseado em árvores
from sklearn.metrics import accuracy_score  # Função para calcular a acurácia (precisão) de um modelo
from sklearn.preprocessing import OrdinalEncoder  # Utilizado para codificar variáveis categóricas em valores numéricos ordenados


In [52]:
# Seleciona as colunas categóricas do DataFrame
object_columns = df_final.select_dtypes(include='object')

# Cria um objeto da classe OrdinalEncoder para realizar a codificação de valores categóricos
encoder = OrdinalEncoder()

# Aplica a codificação ordinal nas colunas categóricas e armazena em um novo DataFrame
codification = pd.DataFrame(encoder.fit_transform(object_columns), columns=object_columns.columns)

# Substitui as colunas categóricas originais do DataFrame com suas versões codificadas
df_final[object_columns.columns] = codification


In [53]:
# Divide o DataFrame 'df_final' em dois conjuntos: 80% para treino/validação e 20% para teste
df_train_valid, df_test = train_test_split(df_final, test_size=0.2)

# Divide o conjunto de treino/validação em treino (75%) e validação (25%)
df_train, df_valid = train_test_split(df_train_valid, test_size=0.25)

In [54]:
# Remove a coluna 'price' do conjunto de treino para obter apenas as features (características) de entrada
features_train = df_train.drop(['price'], axis=1)

# Define a coluna 'price' como o alvo (target) para o conjunto de treino
target_train = df_train['price']

# Remove a coluna 'price' do conjunto de validação para obter apenas as features de entrada
features_valid = df_valid.drop(['price'], axis=1)

# Define a coluna 'price' como o alvo para o conjunto de validação
target_valid = df_valid['price']

# Remove a coluna 'price' do conjunto de teste para obter apenas as features de entrada
features_test = df_test.drop(['price'], axis=1)

# Define a coluna 'price' como o alvo para o conjunto de teste
target_test = df_test['price']


In [55]:
import time
import math
import time
import joblib
from sklearn.metrics import mean_squared_error as mse
import numpy as np

###  Árvore de Decisão


In [56]:
from sklearn.tree import DecisionTreeRegressor

# Inicializando variáveis para armazenar o melhor modelo e resultados
best_model_dt = None
best_result = float('inf')  # Inicializa com infinito para garantir que qualquer resultado será melhor
best_depth = 0

# Medindo o tempo total de execução
start_time = time.time()

# Modelagem da Árvore de Decisão buscando o melhor hiperparâmetro max_depth
for depth in range(1, 50):
    # Criação do modelo com o hiperparâmetro max_depth atual
    model_dt = DecisionTreeRegressor(random_state=12345, max_depth=depth)
    
    # Treinamento do modelo
    model_dt.fit(features_train, target_train) 
    
    # Fazendo previsões no conjunto de validação
    predictions_valid_dt = model_dt.predict(features_valid)
    
    # Calculando RMSE
    result = np.sqrt(mse(target_valid, predictions_valid_dt))
    
    # Verificando se o resultado atual é melhor que o melhor encontrado até agora
    if result < best_result:
        best_model_dt = model_dt  # Armazena o melhor modelo encontrado
        best_result = result  # Atualiza o melhor resultado
        best_depth = depth  # Armazena a profundidade correspondente

# Tempo total de execução
end_time = time.time()
execution_time = end_time - start_time

# Exibindo os resultados finais
print(f"O melhor resultado RMSE da Árvore de Decisão é com o hiperparâmetro max_depth de {best_depth}: {best_result:.4f}")
print(f"Tempo total de execução: {execution_time:.4f} segundos")


O melhor resultado RMSE da Árvore de Decisão é com o hiperparâmetro max_depth de 12: 2830.7926
Tempo total de execução: 19.9122 segundos


### Floresta Aleatória

In [57]:
import time
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

best_model_rf = None
best_result = float('inf')  # Inicializa com infinito para garantir que qualquer resultado será melhor

# Medindo o tempo total de execução
start_time = time.time()

# Criando o modelo Random Forest com parâmetros reduzidos
model_rf = RandomForestRegressor(n_estimators=10, max_depth=5, random_state=12345)

# Treinando o modelo com os dados de treinamento
model_rf.fit(features_train, target_train)

# Fazendo previsões no conjunto de validação
predictions_valid_rf = model_rf.predict(features_valid)

# Calculando RMSE
best_result = np.sqrt(mse(target_valid, predictions_valid_rf))

# Tempo total de execução
end_time = time.time()
execution_time = end_time - start_time

# Exibindo os resultados finais
print(f"O resultado RMSE da Random Forest é: {best_result:.4f}")
print(f"Tempo total de execução: {execution_time:.4f} segundos")

O resultado RMSE da Random Forest é: 2992.5775
Tempo total de execução: 0.8621 segundos


### Regressão Linear

In [58]:
from sklearn.linear_model import LinearRegression

# Inicializando o modelo de Regressão Linear
model_lr = LinearRegression()

# Medindo o tempo total de execução
start_time = time.time()

# Treinando o modelo com os dados de treinamento
model_lr.fit(features_train, target_train)

# Fazendo previsões no conjunto de validação
predictions_valid_lr = model_lr.predict(features_valid)

# Calculando RMSE
rmse = np.sqrt(mse(target_valid, predictions_valid_lr))

# Tempo total de execução
end_time = time.time()
execution_time = end_time - start_time

# Exibindo os resultados finais
print(f"O resultado RMSE da Regressão Linear é: {rmse:.4f}")
print(f"Tempo total de execução: {execution_time:.4f} segundos")

O resultado RMSE da Regressão Linear é: 3462.5451
Tempo total de execução: 0.0676 segundos


### XGBOOST

In [59]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [60]:
# Importando as bibliotecas necessárias
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Inicializando o modelo XGBoost
model_xgb = XGBRegressor(random_state=12345)

# Definindo o espaço de hiperparâmetros para busca
param_grid = {
    'n_estimators': [50, 100],  # Número de árvores
    'max_depth': [3, 5, 7],      # Profundidade máxima das árvores
    'learning_rate': [0.01, 0.1], # Taxa de aprendizado
    'subsample': [0.8]           # Proporção de amostras usadas para treinar cada árvore
}

# Medindo o tempo total de execução
start_time = time.time()

# Usando GridSearchCV para encontrar os melhores hiperparâmetros
grid_search = GridSearchCV(estimator=model_xgb,
                           param_grid=param_grid,
                           scoring='neg_mean_squared_error',
                           cv=3,  # Validação cruzada com 3 folds
                           n_jobs=-1)  # Usar todos os núcleos disponíveis

# Treinando o modelo com os dados de treinamento
grid_search.fit(features_train, target_train)

# Obtendo o melhor modelo encontrado
best_model_xgb = grid_search.best_estimator_

# Fazendo previsões no conjunto de validação usando o melhor modelo
predictions_valid_xgb = best_model_xgb.predict(features_valid)

# Calculando RMSE
rmse = np.sqrt(mse(target_valid, predictions_valid_xgb))

# Tempo total de execução
end_time = time.time()
execution_time = end_time - start_time

# Exibindo os resultados finais
print(f"O resultado RMSE do XGBoost é: {rmse:.4f}")
print(f"Tempo total de execução: {execution_time:.4f} segundos")

O resultado RMSE do XGBoost é: 2708.8131
Tempo total de execução: 7.8551 segundos


### LightGBM

In [61]:
import lightgbm as lgb



# Definindo os parâmetros do modelo LightGBM
params = {
    'boosting_type': 'gbdt',  # Tipo de boosting
    'objective': 'regression',  # Objetivo é regressão
    'metric': 'rmse',  # Métrica de avaliação é RMSE
    'num_leaves': 10,  # Número máximo de folhas em cada árvore
    'learning_rate': 0.05,  # Taxa de aprendizado
    'feature_fraction': 0.5,  # Fração de recursos usada em cada iteração
    'random_state': 12345,  # Estado aleatório para reprodutibilidade
    'verbose': -1  # Nível de verbosidade (-1 para silenciar mensagens)
}

# Criando os datasets de treinamento e validação
lgb_train = lgb.Dataset(features_train, target_train)  # Dataset de treinamento
lgb_eval = lgb.Dataset(features_valid, target_valid, reference=lgb_train)  # Dataset de validação

# Usando callbacks para early stopping
callbacks = [lgb.early_stopping(stopping_rounds=30)]  # Paramos o treinamento se a métrica não melhorar em 10 rodadas

# Medindo o tempo de execução do treinamento e predição
start_time = time.time()  # Inicia a contagem do tempo

#Treinando o modelo LightGBM
model = lgb.train(params,
                  train_set=lgb_train,
                  valid_sets=lgb_eval,
                  num_boost_round=3000,
                  callbacks=callbacks)

# Fazendo previsões no conjunto de validação
predictions_valid_lgbm = model.predict(features_valid)

end_time = time.time()  # Finaliza a contagem do tempo

# Calculando RMSE
rmse = math.sqrt(mse(target_valid, predictions_valid_lgbm))

# Exibindo o resultado do RMSE e o tempo de execução total
print(f'RMSE: {rmse}')
print(f'Tempo total de execução: {end_time - start_time:.4f} segundos')

Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[3000]	valid_0's rmse: 2701.31
RMSE: 2701.307654522641
Tempo total de execução: 3.5440 segundos


## Conclusão geral <a id='end'></a>

Neste projeto, importamos bibliotecas como "pandas" e "numpy" que habitualmente já utilizamos e sklearn para criação do nosso modelo.

Realizamos o pre-processamento, para que não tivesse acontecido algum problema na criação do nosso modelo. O modelo com melhor pontuação no **RMSE** foi o Modelo LightGBM, enquanto o com pior desempenho foi o de Regressão Linear, Testamos o modelo de Árvore de Decisão, Floresta Aleatória e XG Boost.
